In [2]:
!pip3 install tokenizers

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 2.9 MB 1.1 MB/s eta 0:00:01
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
import re
import numpy as np
import pandas as pd
from scipy.sparse import lil_matrix
from collections import Counter, defaultdict
from tokenizers import CharBPETokenizer, Tokenizer

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, accuracy_score, f1_score

# Задание 1

In [13]:
def build_vocab(corpus: str) -> dict:
    """Step 1. Build vocab from text corpus"""

    # Separate each char in word by space and add mark end of token
    tokens = [" ".join(word) for word in corpus.split()]
    
    # Count frequency of tokens in corpus
    vocab = Counter(tokens)  

    return vocab

def get_stats(vocab: dict) -> dict:
    """Step 2. Get counts of pairs of consecutive symbols"""

    pairs = defaultdict(int)
    for word, frequency in vocab.items():
        symbols = word.split()

        # Counting up occurrences of pairs
        for i in range(len(symbols) - 1):
            pairs[symbols[i], symbols[i + 1]] += frequency

    return pairs

def merge_vocab(pair: tuple, v_in: dict) -> dict:
    """Step 3. Merge all occurrences of the most frequent pair"""
    
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    
    for word in v_in:
        # replace most frequent pair in all vocabulary
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]

    return v_out

def get_tokens_from_vocab(vocab):
    tokens_frequencies = defaultdict(int)
    vocab_tokenization = {}
    for word, freq in vocab.items():
        word_tokens = word.split()
        for token in word_tokens:
            tokens_frequencies[token] += freq
        vocab_tokenization[''.join(word_tokens)] = word_tokens
    return tokens_frequencies, vocab_tokenization

def tokenize_text(string, sorted_tokens, unknown_token='</u>'):    
    if string == '':
        return []
    if sorted_tokens == []:
        return [unknown_token]
    string_tokens = []
    for i in range(len(sorted_tokens)):
        token = sorted_tokens[i]
        token_reg = re.escape(token.replace('.', '[.]'))

        matched_positions = [(m.start(0), m.end(0)) for m in re.finditer(token_reg, string)]
        if len(matched_positions) == 0:
            continue
        substring_end_positions = [matched_position[0] for matched_position in matched_positions]

        substring_start_position = 0
        for substring_end_position in substring_end_positions:
            substring = string[substring_start_position:substring_end_position]
            string_tokens += tokenize_text(string=substring, sorted_tokens=sorted_tokens[i + 1:], unknown_token=unknown_token)
            string_tokens += [token]
            substring_start_position = substring_end_position + len(token)
        remaining_substring = string[substring_start_position:]
        string_tokens += tokenize_text(string=remaining_substring, sorted_tokens=sorted_tokens[i + 1:], unknown_token=unknown_token)
        break
    return string_tokens

In [30]:
with open('lenta.txt', 'r', encoding='utf-8') as fhand:
    corpus = fhand.read()
    corpus = corpus.lower()

vocab = build_vocab(corpus)  # Step 1

print('Токены до BPE')
tokens_frequencies, vocab_tokenization = get_tokens_from_vocab(vocab)
print('Токены: {}'.format(tokens_frequencies.keys()))
print('Количество токенов: {}'.format(len(tokens_frequencies.keys())))

Токены до BPE
Токены: dict_keys(['б', 'о', 'и', 'у', 'с', 'п', 'ц', 'к', 'н', 'а', 'д', 'р', 'е', 'з', 'ч', 'л', 'ь', 'т', 'м', 'г', 'в', '.', 'я', ',', 'ш', 'й', 'ю', 'ж', 'ы', '1', '4', 'х', 'щ', '«', '»', '6', '9', '(', '2', '-', ')', 'э', 'ф', '—', 'd', 'a', 's', 'i', 't', 'n', 'e', 'r', 'o', 'f', '!', 'ъ', '7', '5', '3', '№', '…', '–', 'l', 'y', 'm', '8', '0', '"', ':', 'g', 'z', 'u', '$', '%', 'b', 'k', 'w', 'x', 'p', 'h', 'v', "'", 'j', 'ё', '/', 'c', ';', '=', '&', '?', 'q', '*', '@', '+', '’', '·', '“', '”', '•', '>', '_', '|', '[', ']', '\xad', '~', '<', '#', '£', 'ї', 'і', '`'])
Количество токенов: 112


## Эксперимент 1 (N=10, K=3)

Уже прослеживается появление аффиксов, что, как мне кажется, хорошо. Новость токенизирована так, что ничего осмысленного не выделяется

In [33]:
vocab = build_vocab(corpus)  # Step 1

num_merges = 10  # Hyperparameter
k = 3
for i in range(num_merges):

    pairs = get_stats(vocab)  # Step 2

    if not pairs:
        break

    # step 3
    best_top_k = sorted(pairs, key=pairs.get, reverse=True)[:k]
    for best in best_top_k:
        vocab = merge_vocab(best, vocab)

print('Токены после BPE (отсортированные)')
tokens_frequencies, vocab_tokenization = get_tokens_from_vocab(vocab)
sorted_tokens_tuple = sorted(tokens_frequencies.items(), key=lambda item: (len(item[0]), item[1]), reverse=True)
sorted_tokens = [token for (token, freq) in sorted_tokens_tuple]
print('Токены: {}'.format(sorted_tokens))
print('Количество токенов: {}'.format(len(tokens_frequencies.keys())))

Токены после BPE (отсортированные)
Токены: ['ени', 'ст', 'ов', 'ра', 'на', 'по', 'ре', 'ко', 'но', 'ро', 'ен', 'то', 'ни', 'го', 'ли', 'ет', 'ер', 'ль', 'ны', 'ка', 'за', 'не', 'ла', 'ри', 'да', 'во', 'та', 'ми', 'те', 'об', 'и', 'с', 'е', 'о', 'в', 'а', 'д', 'м', 'у', 'л', 'т', 'п', 'к', 'я', 'н', 'р', ',', 'й', 'ч', 'б', 'ы', 'з', '.', 'г', 'х', 'ж', 'ь', 'ц', '"', 'ю', 'ш', 'щ', 'ф', '-', 'э', '0', '1', 'e', '2', 'a', 'o', 's', 'r', '9', 'n', 'i', 't', '5', '3', 'c', '4', 'l', 'ъ', '6', 'm', ')', '(', '8', '7', ':', 'd', 'u', 'p', 'b', 'w', 'f', 'h', 'g', 'k', 'y', 'v', 'x', '%', ';', 'z', 'j', '/', "'", '!', 'q', '&', '№', '?', '$', '—', '«', '»', '–', '+', '@', '“', '”', '·', 'ё', '>', '=', '_', '*', '\xad', '•', '…', '’', '£', '<', 'ї', '|', '[', ']', '~', '#', 'і', '`']
Количество токенов: 142


In [34]:
yandex_news = '''
Губернатор назвал 12 главных достижений Костромской области в этом году
Перед наступлением нового года губернатор Сергей Ситников подвел итоги уходящего года и назвал главные достижения Костромской области, узнал KOSTROMA.TODAY. Сделал он это в Инстаграме.
'''
print(tokenize_text(string=yandex_news.lower(), sorted_tokens=sorted_tokens, unknown_token='</u>'))

['г', 'у', 'б', 'ер', 'на', 'то', 'р', 'на', 'з', 'в', 'а', 'л', '1', '2', 'г', 'ла', 'в', 'ны', 'х', 'д', 'о', 'ст', 'и', 'ж', 'ени', 'й', 'ко', 'ст', 'ро', 'м', 'с', 'ко', 'й', 'об', 'ла', 'ст', 'и', 'в', 'э', 'то', 'м', 'го', 'д', 'у', 'п', 'е', 'ре', 'д', 'на', 'ст', 'у', 'п', 'л', 'ени', 'е', 'м', 'н', 'ов', 'о', 'го', 'го', 'да', 'г', 'у', 'б', 'ер', 'на', 'то', 'р', 'с', 'ер', 'г', 'е', 'й', 'с', 'и', 'т', 'ни', 'к', 'ов', 'по', 'д', 'в', 'е', 'л', 'и', 'то', 'г', 'и', 'у', 'х', 'о', 'д', 'я', 'щ', 'е', 'го', 'го', 'да', 'и', 'на', 'з', 'в', 'а', 'л', 'г', 'ла', 'в', 'ны', 'е', 'д', 'о', 'ст', 'и', 'ж', 'ени', 'я', 'ко', 'ст', 'ро', 'м', 'с', 'ко', 'й', 'об', 'ла', 'ст', 'и', ',', 'у', 'з', 'на', 'л', 'k', 'o', 's', 't', 'r', 'o', 'm', 'a', 't', 'o', 'd', 'a', 'y', 'с', 'д', 'е', 'ла', 'л', 'о', 'н', 'э', 'то', 'в', 'и', 'н', 'ст', 'а', 'г', 'ра', 'м', 'е']


## Эксперимент 2 (N=15, K=5)

Количество аффиксов увеличивается. Новость токенизирована так, что ничего осмысленного не выделяется

In [38]:
vocab = build_vocab(corpus)  # Step 1

num_merges = 15 # Hyperparameter
k = 5
for i in range(num_merges):

    pairs = get_stats(vocab)  # Step 2

    if not pairs:
        break

    # step 3
    best_top_k = sorted(pairs, key=pairs.get, reverse=True)[:k]
    for best in best_top_k:
        vocab = merge_vocab(best, vocab)

print('Токены после BPE (отсортированные)')
tokens_frequencies, vocab_tokenization = get_tokens_from_vocab(vocab)
sorted_tokens_tuple = sorted(tokens_frequencies.items(), key=lambda item: (len(item[0]), item[1]), reverse=True)
sorted_tokens = [token for (token, freq) in sorted_tokens_tuple]
print('Токены: {}'.format(sorted_tokens))
print('Количество токенов: {}'.format(len(tokens_frequencies.keys())))

Токены после BPE (отсортированные)
Токены: ['тель', 'ени', 'про', 'ова', 'ста', 'ско', 'сти', 'что', 'ово', 'ных', 'ски', 'нов', 'ра', 'ст', 'на', 'по', 'но', 'ен', 'ни', 'го', 'ли', 'ет', 'ко', 'ер', 'ов', 'ре', 'пр', 'то', 'ка', 'ро', 'за', 'не', 'ла', 'да', 'об', 'та', 'во', 'ми', 'ль', 'ны', 'ед', 'ва', 'ви', 'со', 'ци', 'ло', 'ся', 'ру', 'ле', 'де', 'си', 'ма', 'ти', 'те', 'от', 'мо', 'ть', 'до', 'ри', 'че', 'ди', 'вы', 'се', 'из', 'па', 'ки', 'бо', 'ин', 'ме', 'ча', 'ля', 'са', 'бы', 'пе', 'сл', 'и', 'е', 'в', 'с', 'у', 'м', 'к', 'о', 'а', 'д', ',', 'н', 'й', 'т', 'я', 'р', 'л', '.', 'г', 'ж', 'з', 'б', 'ы', 'п', 'х', '"', 'ю', 'ч', 'ш', 'щ', 'ф', 'ь', '-', 'ц', 'э', '0', '1', 'e', '2', 'a', 'o', 's', 'r', '9', 'n', 'i', 't', '5', '3', 'c', '4', 'l', 'ъ', '6', 'm', ')', '(', '8', '7', ':', 'd', 'u', 'p', 'b', 'w', 'f', 'h', 'g', 'k', 'y', 'v', 'x', '%', ';', 'z', 'j', '/', "'", '!', 'q', '&', '№', '?', '$', '—', '«', '»', '–', '+', '@', '“', '”', '·', 'ё', '>', '=', '_', '*', '\x

In [39]:
print(tokenize_text(string=yandex_news.lower(), sorted_tokens=sorted_tokens, unknown_token='</u>'))

['г', 'у', 'б', 'ер', 'на', 'то', 'р', 'на', 'з', 'ва', 'л', '1', '2', 'г', 'ла', 'в', 'ных', 'до', 'сти', 'ж', 'ени', 'й', 'ко', 'ст', 'ро', 'м', 'ско', 'й', 'об', 'ла', 'сти', 'в', 'э', 'то', 'м', 'го', 'д', 'у', 'п', 'ер', 'ед', 'на', 'ст', 'у', 'п', 'л', 'ени', 'е', 'м', 'н', 'ово', 'го', 'го', 'да', 'г', 'у', 'б', 'ер', 'на', 'то', 'р', 'с', 'ер', 'г', 'е', 'й', 'си', 'т', 'ни', 'ко', 'в', 'по', 'д', 'в', 'е', 'л', 'и', 'то', 'г', 'и', 'у', 'х', 'о', 'д', 'я', 'щ', 'е', 'го', 'го', 'да', 'и', 'на', 'з', 'ва', 'л', 'г', 'ла', 'в', 'ны', 'е', 'до', 'сти', 'ж', 'ени', 'я', 'ко', 'ст', 'ро', 'м', 'ско', 'й', 'об', 'ла', 'сти', ',', 'у', 'з', 'на', 'л', 'k', 'o', 's', 't', 'r', 'o', 'm', 'a', 't', 'o', 'd', 'a', 'y', 'с', 'де', 'ла', 'л', 'о', 'н', 'э', 'то', 'в', 'ин', 'ста', 'г', 'ра', 'ме']


## Эксперимент 3 (N=20, K=10)

Появляются подтокены, которые можно отнести к корням или частям корней, мне кажется, что это нежелательно. Новость токенизирована так, что ничего осмысленного не выделяется

In [36]:
vocab = build_vocab(corpus)  # Step 1

num_merges = 20 # Hyperparameter
k = 10
for i in range(num_merges):

    pairs = get_stats(vocab)  # Step 2

    if not pairs:
        break

    # step 3
    best_top_k = sorted(pairs, key=pairs.get, reverse=True)[:k]
    for best in best_top_k:
        vocab = merge_vocab(best, vocab)

print('Токены после BPE (отсортированные)')
tokens_frequencies, vocab_tokenization = get_tokens_from_vocab(vocab)
sorted_tokens_tuple = sorted(tokens_frequencies.items(), key=lambda item: (len(item[0]), item[1]), reverse=True)
sorted_tokens = [token for (token, freq) in sorted_tokens_tuple]
print('Токены: {}'.format(sorted_tokens))
print('Количество токенов: {}'.format(len(tokens_frequencies.keys())))

Токены после BPE (отсортированные)
Токены: ['россии', 'сообщ', 'ского', 'росси', 'ности', 'ствен', 'ения', 'ного', 'ение', 'пред', 'тель', 'стра', 'кото', 'пере', 'ской', 'ства', 'ется', 'рова', 'ство', 'мини', 'ново', 'ских', 'росс', 'про', 'ста', 'ова', 'что', 'ных', 'пре', 'при', 'ени', 'ной', 'сто', 'сти', 'ции', 'ово', 'ент', 'пра', 'нов', 'под', 'это', 'ски', 'раз', 'ные', 'ком', 'вер', 'ров', 'ель', 'кон', 'ско', 'тер', 'его', 'ков', 'мен', 'ден', 'ный', 'сле', 'гра', 'как', 'лен', 'дел', 'дол', 'ния', 'ным', 'сть', 'гла', 'сов', 'пер', 'ска', 'сво', 'ств', 'ить', 'спо', 'тел', 'мер', 'так', 'нии', 'дет', 'дер', 'ном', 'тов', 'сту', 'все', 'слу', 'жен', 'тор', 'общ', 'на', 'по', 'ра', 'ли', 'ни', 'ре', 'за', 'ет', 'не', 'да', 'но', 'ро', 'го', 'ст', 'та', 'ка', 'ми', 'ла', 'ко', 'ен', 'ви', 'ль', 'во', 'ов', 'об', 'ло', 'то', 'ру', 'мо', 'ма', 'ва', 'ти', 'бо', 'де', 'со', 'ин', 'ер', 'че', 'ся', 'из', 'вы', 'до', 'ки', 'па', 'ци', 'ть', 'ме', 'от', 'ри', 'ди', 'ле', 'ча', 'ит',

In [37]:
print(tokenize_text(string=yandex_news.lower(), sorted_tokens=sorted_tokens, unknown_token='</u>'))

['гу', 'б', 'ер', 'на', 'тор', 'на', 'з', 'ва', 'л', '1', '2', 'гла', 'в', 'ных', 'до', 'сти', 'ж', 'ени', 'й', 'ко', 'ст', 'ро', 'м', 'ской', 'об', 'ла', 'сти', 'в', 'это', 'м', 'го', 'ду', 'пере', 'д', 'на', 'сту', 'п', 'л', 'ение', 'м', 'ново', 'го', 'го', 'да', 'гу', 'б', 'ер', 'на', 'тор', 'с', 'ер', 'г', 'ей', 'с', 'ит', 'ни', 'ков', 'под', 'ве', 'л', 'и', 'то', 'ги', 'у', 'х', 'од', 'я', 'щ', 'его', 'го', 'да', 'и', 'на', 'з', 'ва', 'л', 'гла', 'в', 'ные', 'до', 'сти', 'ж', 'ения', 'ко', 'ст', 'ро', 'м', 'ской', 'об', 'ла', 'сти', ',', 'у', 'з', 'на', 'л', 'k', 'o', 's', 't', 'r', 'o', 'm', 'a', 't', 'o', 'd', 'a', 'y', 'с', 'дел', 'а', 'л', 'он', 'это', 'в', 'ин', 'ста', 'гра', 'ме']


## Выводы

Мне кажется, лучший результат показывает второй эксперимент - там еще не выделяются части корней слов типа "росси", "сообщ", но уже есть частотные аффиксы типа "тель", "ова", "ени" и "про". 

# Задание 2

In [3]:
df = pd.read_csv('dataset_ok.csv')
df['text'].to_csv('dataset_ok_text.csv', index=False)

subtoken = CharBPETokenizer()
subtoken.train('dataset_ok_text.csv', vocab_size=3000)

subtoken.save('subtoken_CBPE')
subtoken = Tokenizer.from_file('subtoken_CBPE')

In [4]:
data = df['text'].tolist()
N = len(data)
K = len(subtoken.get_vocab())

X = lil_matrix((N, K))
X_idf = lil_matrix((N, K))
for i, doc in enumerate(data):
    all_tokens = subtoken.encode(doc).ids
    for token in all_tokens:
        if X_idf[i, token] == 0:
            X_idf[i, token] = 1
        X[i, token] += 1
        
idf = pd.Series(X_idf.sum(axis=0).tolist()[0])
idf = idf.apply(lambda x: np.log((1 + idf.shape[0]) / (1 + x)) + 1)
X = X.multiply(lil_matrix(idf.tolist()))

In [5]:
X

<71987x3000 sparse matrix of type '<class 'numpy.float64'>'
	with 1696771 stored elements in Compressed Sparse Row format>

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, df['label'], test_size=0.3, stratify=df['label'])

In [12]:
clf = LogisticRegression(n_jobs=-1, verbose=1, max_iter=100)

In [13]:
kfold = StratifiedKFold(n_splits=8)

In [14]:
cross_val_score(clf, X, df['label'], cv=kfold, scoring='f1_macro')

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    3.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    3.6s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    3.8s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    3.8s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    3.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    3.8s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1

array([0.72314321, 0.7168705 , 0.72426396, 0.69513566, 0.75557527,
       0.70169099, 0.75323991, 0.7220809 ])

In [15]:
cross_val_score(clf, X, df['label'], cv=kfold, scoring='f1_micro')

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    4.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    3.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    3.9s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    3.9s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    3.6s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    3.9s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1

array([0.93399267, 0.93254806, 0.93343705, 0.93376306, 0.93954212,
       0.93187375, 0.93365192, 0.93231829])

In [16]:
clf.fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    3.1s finished


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=-1, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=1,
                   warm_start=False)

In [17]:
y_pred = clf.predict(X_test)

In [18]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

      INSULT       0.82      0.74      0.78      2575
      NORMAL       0.95      0.97      0.96     18317
   OBSCENITY       0.55      0.40      0.47       205
      THREAT       0.68      0.63      0.65       500

    accuracy                           0.93     21597
   macro avg       0.75      0.69      0.72     21597
weighted avg       0.93      0.93      0.93     21597



## Вывод

Результат классификации по метрике F1 неплохой, расхождение в значениях macro и micro связаны с дисбалансом в классах